In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
import seaborn as sns
import pandas as pd
import os
import matplotlib.pyplot as plt


In [ ]:
excel_original = pd.read_excel("/content/drive/MyDrive/용접_데이터셋/201020_TestData/20201020_6.2_1P_N.xlsx")

In [ ]:
#excel_original: 모든 파일 통합 데이터
excel=excel_original.drop(['Unnamed: 6','Unnamed: 14','Unnamed: 19','용접선 이탈 (벽)','용접선 이탈'],axis=1)#고려필요 없음(read_me)
excel=excel.drop(['용접속도','송급속도','전압보정','위빙(hz)','위빙(좌)','위빙(우)'],axis=1)#고려필요 없음(read_me) 질문필요
excel=excel.drop(excel[(excel['Start']==1.0)|(excel['End']==1.0)].index)#Start, End에 해당=> 제거(read_me)
excel=excel.drop(['Start','End'],axis=1)
excel=excel.drop(['Pass','형상','두께(1)','두께(2)'],axis=1)#다 일정한 값을 가짐
excel=excel.drop(['WIRESPEED','WINDSPEED','WINDANGLE'],axis=1)#read_me 조건에서 처리 질문필요

In [ ]:
print(excel.isnull().sum())
print(sum(excel.isnull().sum(axis=1)==4))

In [ ]:
excel

<bound method DataFrame.info of       WELDTIME  CURRENT  VOLTAGE   GAS   정상  언더컷   기공   용락  오버랩
141       1.42    384.0     30.6  32.5  1.0  0.0  0.0  0.0  0.0
142       1.43    339.1     31.8  32.5  1.0  0.0  0.0  0.0  0.0
143       1.44    481.5     29.6  32.5  1.0  0.0  0.0  0.0  0.0
144       1.45    261.7     29.7  32.4  1.0  0.0  0.0  0.0  0.0
145       1.46    433.4     30.7  32.4  1.0  0.0  0.0  0.0  0.0
...        ...      ...      ...   ...  ...  ...  ...  ...  ...
3947     39.48    304.7     30.9  23.1  1.0  0.0  0.0  0.0  0.0
3948     39.49    434.4     30.5  23.3  1.0  0.0  0.0  0.0  0.0
3949     39.50    423.8     30.4  23.2  1.0  0.0  0.0  0.0  0.0
3950     39.51    364.3     30.0  23.1  1.0  0.0  0.0  0.0  0.0
3951     39.52    455.7     31.3  23.1  1.0  0.0  0.0  0.0  0.0

[3811 rows x 9 columns]>

In [ ]:
excel_a=excel.agg(['WELDTIME','CURRENT','VOLTAGE','GAS'],axis=1)
excel_b=excel.agg(['정상','언더컷','기공','용락', '오버랩'],axis=1)

In [ ]:
excel_a

,WELDTIME,CURRENT,VOLTAGE,GAS
141,1.42,384.0,30.6,32.5
142,1.43,339.1,31.8,32.5
143,1.44,481.5,29.6,32.5
144,1.45,261.7,29.7,32.4
145,1.46,433.4,30.7,32.4
...,...,...,...,...
3947,39.48,304.7,30.9,23.1
3948,39.49,434.4,30.5,23.3
3949,39.50,423.8,30.4,23.2
3950,39.51,364.3,30.0,23.1


In [ ]:
excel_b

,정상,언더컷,기공,용락,오버랩
141,1.0,0.0,0.0,0.0,0.0
142,1.0,0.0,0.0,0.0,0.0
143,1.0,0.0,0.0,0.0,0.0
144,1.0,0.0,0.0,0.0,0.0
145,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
3947,1.0,0.0,0.0,0.0,0.0
3948,1.0,0.0,0.0,0.0,0.0
3949,1.0,0.0,0.0,0.0,0.0
3950,1.0,0.0,0.0,0.0,0.0


In [ ]:
#이전 행의 백분율 변경
excel_a=excel_a.pct_change()
excel_a=excel_a.fillna(0)
excel_a

,WELDTIME,CURRENT,VOLTAGE,GAS
141,0.000000,0.000000,0.000000,0.000000
142,0.007042,-0.116927,0.039216,0.000000
143,0.006993,0.419935,-0.069182,0.000000
144,0.006944,-0.456490,0.003378,-0.003077
145,0.006897,0.656095,0.033670,0.000000
...,...,...,...,...
3947,0.000253,-0.276762,0.003247,-0.004310
3948,0.000253,0.425665,-0.012945,0.008658
3949,0.000253,-0.024401,-0.003279,-0.004292
3950,0.000253,-0.140396,-0.013158,-0.004310


In [ ]:
excel_final=pd.concat([excel_a,excel_b],axis=1)
excel_final

,WELDTIME,CURRENT,VOLTAGE,GAS,정상,언더컷,기공,용락,오버랩
141,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0
142,0.007042,-0.116927,0.039216,0.000000,1.0,0.0,0.0,0.0,0.0
143,0.006993,0.419935,-0.069182,0.000000,1.0,0.0,0.0,0.0,0.0
144,0.006944,-0.456490,0.003378,-0.003077,1.0,0.0,0.0,0.0,0.0
145,0.006897,0.656095,0.033670,0.000000,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
3947,0.000253,-0.276762,0.003247,-0.004310,1.0,0.0,0.0,0.0,0.0
3948,0.000253,0.425665,-0.012945,0.008658,1.0,0.0,0.0,0.0,0.0
3949,0.000253,-0.024401,-0.003279,-0.004292,1.0,0.0,0.0,0.0,0.0
3950,0.000253,-0.140396,-0.013158,-0.004310,1.0,0.0,0.0,0.0,0.0


In [ ]:
excel_final2=pd.concat([excel1,excel2,excel3,excel4,excel5,excel6,excel7,excel8,excel9,excel10,excel11,excel12],axis=0)
excel_final2

,WELDTIME,CURRENT,VOLTAGE,GAS,정상,언더컷,기공,용락,오버랩
304,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0
305,0.003279,0.050049,0.026471,0.000000,1.0,0.0,0.0,0.0,0.0
306,0.003268,-0.042523,-0.008596,-0.003717,1.0,0.0,0.0,0.0,0.0
307,0.003257,0.028062,0.011561,-0.003731,1.0,0.0,0.0,0.0,0.0
308,0.003247,-0.107287,0.005714,0.000000,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
3947,0.000253,-0.276762,0.003247,-0.004310,1.0,0.0,0.0,0.0,0.0
3948,0.000253,0.425665,-0.012945,0.008658,1.0,0.0,0.0,0.0,0.0
3949,0.000253,-0.024401,-0.003279,-0.004292,1.0,0.0,0.0,0.0,0.0
3950,0.000253,-0.140396,-0.013158,-0.004310,1.0,0.0,0.0,0.0,0.0
